# Example Evaluation Code

This notebook will be very __similar__ to the code I use to evaluate your results - it is provided for __your convenience__ so that you can use it to evaluate your preprocessing results at any time before your __final submission__.

Please note that the results here will __NOT__ be the same as my evaluation results.

Let's start with loading the required packages.

In [1]:
# import required package for data handling
import pandas as pd
import numpy as np

# import required packages for splitting data
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# import required packages for evaluating models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# import `logistic regression` model
from sklearn.linear_model import LogisticRegression

Next you should load __your__ data. In this case, I am using a sample dataset (`GroupX.csv`) which contains 6 predictors (`X1 - X6`) and two target variables (`Y1, Y2`).

Please make sure you change the data to your __OWN__ dataset when using this code.

__NOTE__:
1. Your dataset maybe very different from the sample dataset.
2. Please follow this structure when submitting your dataset.

In [2]:
data = pd.read_csv('./Datasets/I_ts_IQR.csv', header=0)
data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Y1,Y2
0,0,0.049017,0.0,0.661154,-0.669216,0.175548,0.315491,-0.176365,0.165719,-0.191238,1.046536,0.014029,1.244491,1.003035,0.0,-0.806226,0,1
1,1,1.839967,0.0,-0.634992,-0.924771,0.237492,-0.109993,-1.312510,0.836240,-0.212449,-0.955533,2.061600,-0.752345,1.003035,0.0,-0.806226,1,0
2,2,-0.503911,0.0,0.405772,-1.217767,0.123885,1.435496,2.259363,-1.018075,-0.001480,-0.955533,-0.303770,-0.752345,1.003035,0.0,-0.806226,1,0
3,3,1.259325,0.0,0.384635,-1.192655,-1.267028,1.415545,2.259363,-0.640778,0.765716,-0.955533,-0.350153,-0.752345,1.003035,0.0,-0.806226,1,1
4,4,-0.696399,0.0,-1.293151,1.547529,-0.583199,-0.534020,-0.408541,0.471972,-0.058150,1.046536,-0.072424,0.587173,-0.996974,0.0,1.240347,0,1


Checking your data types and make sure it follows the data dictionary would be an important step, you can do that using the `.dtypes` attribute.

__NOTE__: all __continuous__ faetures will be in `float64` data type, and all __categorical__ features will be in `int64` data type (given you already coded (per __suggest task \#6__ in the competition document) them).

In [3]:
data.dtypes

Unnamed: 0      int64
0             float64
1             float64
2             float64
3             float64
4             float64
5             float64
6             float64
7             float64
8             float64
9             float64
10            float64
11            float64
12            float64
13            float64
14            float64
Y1              int64
Y2              int64
dtype: object

Now you need to specify your targets and predictors. __NOTE__ we have two targets here (`Y1, Y2`).

In [4]:
y1 = data.Y1
y2 = data.Y2

Check the shape of the data.

In [5]:
data.shape

(660, 18)

It is very possible that you will use different sets of the predictors for `Y1` and `Y2`. Now let's define them.

First, let's define predictors for `Y1` - which will be the first 5 features in `data`.

In [6]:
cols = list(data.columns)
# first 5 features 
cols[:-2]

['Unnamed: 0',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14']

Use below code to select the first 5 features as predictors for `Y1`.

In [7]:
datay1 = data[["3","4","6","7","12","14"]]

In [8]:
#datay2 = data[["1","2","6","7","8","10","11"]]
datay2 = data[["2","7","10","11"]]

In [9]:
#datay1 = data.drop(data.columns[12], axis = 1)

In [10]:
#datay1[datay1.columns[0:15]]

In [11]:

predictors_y1 = datay1

predictors_y1.head()

,3,4,6,7,12,14
0,-0.669216,0.175548,-0.176365,0.165719,1.003035,-0.806226
1,-0.924771,0.237492,-1.312510,0.836240,1.003035,-0.806226
2,-1.217767,0.123885,2.259363,-1.018075,1.003035,-0.806226
3,-1.192655,-1.267028,2.259363,-0.640778,1.003035,-0.806226
4,1.547529,-0.583199,-0.408541,0.471972,-0.996974,1.240347


Upon investigation of the data, we know we have __six__ features (`X1 - X6`) predicting `Y2`. Use similar code (as below) to select them.

In [12]:
predictors_y2 = datay2
predictors_y2.head()

,2,7,10,11
0,0.661154,0.165719,0.014029,1.244491
1,-0.634992,0.836240,2.061600,-0.752345
2,0.405772,-1.018075,-0.303770,-0.752345
3,0.384635,-0.640778,-0.350153,-0.752345
4,-1.293151,0.471972,-0.072424,0.587173


Below is the key part of this notebook - which generates a `logistic regression` model to predict `Y1`/`Y2`.

The code works this way:

1. We generate two lists `f1_score_lst` and `auc_lst` to store f1_score and AUC from each of the `10` runs of the model;
2. Define model:
    1. We define a `LogisticRegression()` model;
    
    2. We split predictors (`predictors_y1`) and target `y1` to training (80%) and testing (20%);
    
    3. We fit the model `clf` to the training data, then use it to predict on the testing data;
    
    4. We also defined a `10-fold cross validation` to make sure our model do not overfit - see [here](https://scikit-learn.org/stable/modules/cross_validation.html) for more info;
    
    5. We append the f1_score and AUC of current model to the lists (`f1_score_lst` and `auc_lst`) we defined earlier.
  
3. Print out average f1_score and AUC for all 10 runs;
4. Print out average average accuracy from cross validation
5. Print out confusion matrix and classification report for the __last__ model.

__NOTE__: Step 3 provides the evaluation results we need; step 4 - 5 can be used to verify the results from step 3.

In [13]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    #Model building
    clf = LogisticRegression()
    X1_train, X1_test, y1_train, y1_test = train_test_split(predictors_y1, y1, test_size=0.2, random_state=123)
    clf.fit(X1_train, y1_train)

    y1_pred = clf.predict(X1_test)

    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf, X1_train, y1_train, cv=kfold, scoring=scoring)

    

    
    #calculate f1-score and AUC
    
    clf_roc_auc = roc_auc_score(y1_test, y1_pred)
    f1_score_lst.append(precision_recall_fscore_support(y1_test, y1_pred, average='weighted')[2])
    auc_lst.append(clf_roc_auc)


print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

#result=logit_model.fit()
confusion_matrix_y1 = confusion_matrix(y1_test, y1_pred)


#print(result.summary())
print('Accuracy of classifier on test set: {:.2f}'.format(clf.score(X1_test, y1_test)))

print("10-fold cross validation average accuracy of classifier: %.3f" % (results.mean()))

print('Confusion Matrix for Logistic Regression Classfier:')
print(confusion_matrix_y1)

print('Classification Report for Logistic Regression Classfier:')
print(classification_report(y1_test, y1_pred))


F1 0.5697; AUC 0.5833 
Accuracy of classifier on test set: 0.57
10-fold cross validation average accuracy of classifier: 0.583
Confusion Matrix for Logistic Regression Classfier:
[[39 39]
 [18 36]]
Classification Report for Logistic Regression Classfier:
             precision    recall  f1-score   support

          0       0.68      0.50      0.58        78
          1       0.48      0.67      0.56        54

avg / total       0.60      0.57      0.57       132



Below code are used to evaluate model toward `Y2`. It is very similar to the code above - key difference is that `Y2` is imbalanced - so I wrote some code (under `# Begin oversampling`) to deal with that.

In [14]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    #Model building
    clf1 = LogisticRegression()

    
    # Splitting data into testing and training
    X2_train, X2_test, y2_train, y2_test = train_test_split(predictors_y2, y2, test_size=0.2, random_state=123)
    
    # Begin oversampling
    oversample = pd.concat([X2_train,y2_train],axis=1)
    max_size = oversample['Y2'].value_counts().max()
    lst = [oversample]
    for class_index, group in oversample.groupby('Y2'):
        lst.append(group.sample(max_size-len(group), replace=True))
    X2_train = pd.concat(lst)
    y2_train=pd.DataFrame.copy(X2_train['Y2'])
    del X2_train['Y2']
    
    # fitting model on oversampled data
    clf1.fit(X2_train, y2_train)
    
    y2_pred = clf1.predict(X2_test)
    
    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=123)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf1, X2_train, y2_train, cv=kfold, scoring=scoring)
    
    #calculate f1-score and AUC
    
    clf1_roc_auc = roc_auc_score(y2_test, y2_pred)
    
    
    #calculate average f1-score and AUC
    f1_score_lst.append(precision_recall_fscore_support(y2_test, y2_pred, average='weighted')[2])
    auc_lst.append(clf1_roc_auc)
    
    
print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

confusion_matrix_y2 = confusion_matrix(y2_test, y2_pred)


print('Accuracy of classifier on test set: {:.3f}'.format(clf1.score(X2_test, y2_test)))

print("10-fold cross validation average accuracy of clf1: %.3f" % (results.mean()))

print('Confusion Matrix for Classfier:')
print(confusion_matrix_y2)

print('Classification Report for Classfier:')
print(classification_report(y2_test, y2_pred))


F1 0.6496; AUC 0.6682 
Accuracy of classifier on test set: 0.614
10-fold cross validation average accuracy of clf1: 0.595
Confusion Matrix for Classfier:
[[27  9]
 [42 54]]
Classification Report for Classfier:
             precision    recall  f1-score   support

          0       0.39      0.75      0.51        36
          1       0.86      0.56      0.68        96

avg / total       0.73      0.61      0.63       132

